In [2]:
import os
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
import pymysql.cursors
import pandas_profiling

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

# 기본 폰트 설정
layout_font = {'font':dict(size=18,color='#60606e',family='Franklin Gothic' )}


## Database Connection (Data Mart)

In [216]:
connection = pymysql.connect(host='localhost',
        user='root',
        password='1234',
        db='datamart',
        charset='utf8',
        cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # Read a single record (sql)
        sql = "SELECT * FROM dataset;"
        cursor.execute(sql) # cursor 실행
        result = cursor.fetchall()
        df = pd.DataFrame(result) # dateframe에 result 저장
        df.sort_values(by=['시간대구분' ,'행정동명'],ascending=True,inplace=True)
finally:
    connection.close()

In [217]:
sub_query = df.query('(행정동명 in ["이태원1동","이태원2동"])&(시간대구분 in [18,20])') #&(년==2018)
# sub_query.sort_values(by=['시간대구분' ,'행정동명'],ascending=True,inplace=True)

In [218]:
sub_query

,시간대구분,행정동코드,총생활인구수,시도명,시군구명,행정동명,년,월
299,18,11170650,1613,서울,용산구,이태원1동,2017,1
683,18,11170650,1684,서울,용산구,이태원1동,2017,2
1067,18,11170650,1802,서울,용산구,이태원1동,2017,3
1451,18,11170650,2126,서울,용산구,이태원1동,2017,4
1835,18,11170650,1642,서울,용산구,이태원1동,2017,5
...,...,...,...,...,...,...,...,...
16844,20,11170660,751,서울,용산구,이태원2동,2020,8
17196,20,11170660,714,서울,용산구,이태원2동,2020,9
17612,20,11170660,748,서울,용산구,이태원2동,2020,10
17996,20,11170660,654,서울,용산구,이태원2동,2020,11


In [212]:
sub_query12 = sub_query.query('(시간대구분)==18')
sub_query13 = sub_query.query('(시간대구분)==20')

UndefinedVariableError: name '시간대구분' is not defined

In [ ]:
sub_query = sub_query[['년','월','행정동명']]

In [213]:
sub_query.drop_duplicates(inplace=True)

In [214]:
sub_query['평균유동인구수'] = (sub_query12['총생활인구수'].values + sub_query13['총생활인구수'].values) // 2

ValueError: Length of values (0) does not match length of index (96)

In [198]:
fig = px.line(sub_query,
                x='월', y='평균유동인구수',#size='pop'
                color='행정동명',
                facet_col='년', facet_col_wrap=2)
fig.show()

In [26]:
fig = px.line(sub_query,
                x='월', y='평균생활인구수',#size='pop',
                color='행정동명',
                facet_col='월', facet_col_wrap=4)

fig.show()

In [23]:
sub_query1 = df.query('(행정동명 in ["이태원1동"])&(년==2020)&(월==7)') #
fig = px.line(sub_query, x=sub_query['월'], y=sub_query['평균생활인구수'], color='행정동명')
fig.show()

In [32]:
sub_query1.head(3)

,시간대구분,행정동코드,총생활인구수,시도명,시군구명,행정동명,년,월
16415,0,11170650,554,서울,용산구,이태원1동,2020,7
16405,1,11170650,603,서울,용산구,이태원1동,2020,7
16395,2,11170650,897,서울,용산구,이태원1동,2020,7


In [66]:
fig = px.line(sub_query, x=sub_query['시간대구분'], y=sub_query['총생활인구수'], color='행정동명')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sub_query['시간대구분'],y=sub_query['총생활인구수'],
                            mode='lines+markers+text',
                            #name = 'lines+makers',
                            text = list(sub_query['총생활인구수']),
                            textposition= 'top center',
                            texttemplate='%{text:.3s}'

                            ))

fig.show()

In [ ]:
fig = px.bar(sub_query, x=sub_query['시간대구분'] ,y=sub_query['총생활인구수'] , text=sub_query['총생활인구수'])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.add_trace(go.Scatter(x=sub_query['시간대구분'],y=sub_query['총생활인구수'],
                            mode='lines+markers',name = 'lines+markers'
                            ))

fig.show()